<a href="https://colab.research.google.com/github/KeerthanaNarayan/Contrastive_Learning_for_Fall_detection/blob/main/Supervised_Learning_for_Fall_detection_ARCO_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import random
def chooseData(org_data, no_of_points):
    # Create a copy of the original data to avoid modifying it
    shuffled_data = org_data.copy()

    # Randomly shuffle the data
    np.random.shuffle(shuffled_data)
    shuffled_data = shuffled_data.tolist()

    reduced_data = random.sample(shuffled_data, no_of_points)

    # Split the data and labels
    data, labels = np.array(reduced_data)[:, :-1], np.array(reduced_data)[:, -1]

    return data, labels

In [44]:
import os
import zipfile
import pandas as pd

zip_path = 'fall-dataset-raw.zip' #zip_path = 'fall-dataset-all.zip'

# Extract the CSV files from the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    csv_files = [file for file in zip_ref.namelist() if file.endswith('.csv')]
    zip_ref.extractall(members=csv_files)

# Read and concatenate the extracted CSV files into a DataFrame
data = pd.concat([pd.read_csv(file, encoding='latin-1') for file in csv_files], ignore_index=True)
accelerometer_data = data[["Acc(X)", "Acc(Y)", "Acc(Z)", "Rot(X)", "Rot(Y)", "Rot(Z)", "Pitch", "Roll", "Yaw", "Timestamp"]].values
labels = list(data["Fall"])#.values
# # Standardize the data
# scaler = StandardScaler()
# standardized_data = scaler.fit_transform(accelerometer_data)



In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataset
# train_data = pd.read_csv("raw182_Training_Relabeled_Auto_25.csv")

# # Separate features and labels
# raw_data = data[[" ms_accelerometer_x", " ms_accelerometer_y", " ms_accelerometer_z"]].values
# labels = data["outcome"]

num_training_points = 8000
num_testing_points = 2000

data_with_labels = np.column_stack((accelerometer_data, labels))

# Split the data into training and testing sets with an 80:20 ratio
split = int(0.8 * len(data_with_labels))
train_data = data_with_labels[:split]
test_data = data_with_labels[split:]

print(train_data)
print(test_data)



[[ 3.69726562e+00  5.96679688e-01  1.78906250e+00 ...  3.56835938e+02
   1.61528564e+12  0.00000000e+00]
 [ 1.58203125e-01  6.03515625e-01 -1.90917969e-01 ...  3.56835938e+02
   1.61528564e+12  0.00000000e+00]
 [ 3.85742188e-01  4.61914062e-01 -9.37500000e-02 ...  6.41827154e+00
   1.61528564e+12  0.00000000e+00]
 ...
 [ 1.00585938e+00  3.22265625e-02  1.78710938e-01 ...  3.28777557e+02
   1.61254540e+12  0.00000000e+00]
 [ 1.00488281e+00  3.07617188e-02  1.78710938e-01 ...  3.28762085e+02
   1.61254540e+12  0.00000000e+00]
 [ 1.00146484e+00  2.97851562e-02  1.77246094e-01 ...  3.28750946e+02
   1.61254540e+12  0.00000000e+00]]
[[ 1.00048828e+00  3.17382812e-02  1.79687500e-01 ...  3.28738281e+02
   1.61254540e+12  0.00000000e+00]
 [ 1.00439453e+00  3.22265625e-02  1.80664062e-01 ...  3.28738281e+02
   1.61254540e+12  0.00000000e+00]
 [ 1.00341797e+00  3.12500000e-02  1.79199219e-01 ...  3.28698181e+02
   1.61254540e+12  0.00000000e+00]
 ...
 [-3.07617188e-01  1.66015625e-02  9.5996093

In [46]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# # Initialize and train the model
# model = RandomForestClassifier(random_state=42)
# model.fit(X_train_scaled, y_train_chosen)

# # Make predictions
# y_pred = model.predict(X_test_scaled)

# # Calculate accuracy
# accuracy = accuracy_score(y_test_chosen, y_pred)
# print(f"Accuracy: {accuracy:.2f}")


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score
import numpy as np

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Set the number of iterations
num_iterations = 3
accuracies = []
recalls = []


for iteration in range(num_iterations):
    #Choose different data for training and testing in every iteration
    X_train_chosen, y_train_chosen = chooseData(train_data, num_training_points)
    X_test_chosen, y_test_chosen = chooseData(test_data, num_testing_points)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_chosen)
    X_test_scaled = scaler.transform(X_test_chosen)

    # Initialize and train the model
    model.fit(X_train_scaled, y_train_chosen)

    # Make predictions
    y_pred = model.predict(X_test_scaled)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_chosen, y_pred)
    accuracies.append(accuracy)

    # Calculate recall
    recall = recall_score(y_test_chosen, y_pred)
    recalls.append(recall)

    print(f"Iteration {iteration+1} - Accuracy: {accuracy:.2f}, Recall: {recall:.2f}")

average_accuracy = np.mean(accuracies)
average_recall = np.mean(recalls)

print(f"Average Accuracy across {num_iterations} iterations: {average_accuracy:.2f}")
print(f"Average Recall across {num_iterations} iterations: {average_recall:.2f}")

Iteration 1 - Accuracy: 0.93, Recall: 0.95
Iteration 2 - Accuracy: 0.94, Recall: 0.94
Iteration 3 - Accuracy: 0.94, Recall: 0.95
Average Accuracy across 3 iterations: 0.93
Average Recall across 3 iterations: 0.95


In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the logistic regression model
logreg_model = LogisticRegression(random_state=42)

# Set the number of iterations
num_iterations = 3
accuracies = []
recalls = []

for iteration in range(num_iterations):
    #Choose different data for training and testing in every iteration
    X_train_chosen, y_train_chosen = chooseData(train_data, num_training_points)
    X_test_chosen, y_test_chosen = chooseData(test_data, num_testing_points)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_chosen)
    X_test_scaled = scaler.transform(X_test_chosen)

    # train the model
    logreg_model.fit(X_train_scaled, y_train_chosen)

    # Make predictions
    y_pred = logreg_model.predict(X_test_scaled)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_chosen, y_pred)
    accuracies.append(accuracy)

    # Calculate recall
    recall = recall_score(y_test_chosen, y_pred)
    recalls.append(recall)

    print(f"Iteration {iteration+1} - Accuracy: {accuracy:.2f}, Recall: {recall:.2f}")

average_accuracy = np.mean(accuracies)
average_recall = np.mean(recalls)

print(f"Average Accuracy across {num_iterations} iterations: {average_accuracy:.2f}")
print(f"Average Recall across {num_iterations} iterations: {average_recall:.2f}")

Iteration 1 - Accuracy: 0.92, Recall: 0.89
Iteration 2 - Accuracy: 0.92, Recall: 0.90
Iteration 3 - Accuracy: 0.92, Recall: 0.89
Average Accuracy across 3 iterations: 0.92
Average Recall across 3 iterations: 0.89


In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Initialize and train the KNN model
k = 5  # Number of neighbors
knn_model = KNeighborsClassifier(n_neighbors=k)

# Set the number of iterations
num_iterations = 3
accuracies = []
recalls = []

for iteration in range(num_iterations):
    #Choose different data for training and testing in every iteration
    X_train_chosen, y_train_chosen = chooseData(train_data, num_training_points)
    X_test_chosen, y_test_chosen = chooseData(test_data, num_testing_points)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_chosen)
    X_test_scaled = scaler.transform(X_test_chosen)

    # train the model
    knn_model.fit(X_train_scaled, y_train_chosen)

    # Make predictions
    y_pred = knn_model.predict(X_test_scaled)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_chosen, y_pred)
    accuracies.append(accuracy)

    # Calculate recall
    recall = recall_score(y_test_chosen, y_pred)
    recalls.append(recall)

    print(f"Iteration {iteration+1} - Accuracy: {accuracy:.2f}, Recall: {recall:.2f}")

average_accuracy = np.mean(accuracies)
average_recall = np.mean(recalls)

print(f"Average Accuracy across {num_iterations} iterations: {average_accuracy:.2f}")
print(f"Average Recall across {num_iterations} iterations: {average_recall:.2f}")

Iteration 1 - Accuracy: 0.92, Recall: 0.89
Iteration 2 - Accuracy: 0.91, Recall: 0.89
Iteration 3 - Accuracy: 0.87, Recall: 0.89
Average Accuracy across 3 iterations: 0.90
Average Recall across 3 iterations: 0.89


In [53]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Initialize and train the Naive Bayes model
nb_model = GaussianNB()

# Set the number of iterations
num_iterations = 3
accuracies = []
recalls = []

for iteration in range(num_iterations):
    #Choose different data for training and testing in every iteration
    X_train_chosen, y_train_chosen = chooseData(train_data, num_training_points)
    X_test_chosen, y_test_chosen = chooseData(test_data, num_testing_points)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_chosen)
    X_test_scaled = scaler.transform(X_test_chosen)

    # train the model
    nb_model.fit(X_train_scaled, y_train_chosen)

    # Make predictions
    y_pred = nb_model.predict(X_test_scaled)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_chosen, y_pred)
    accuracies.append(accuracy)

    # Calculate recall
    recall = recall_score(y_test_chosen, y_pred)
    recalls.append(recall)

    print(f"Iteration {iteration+1} - Accuracy: {accuracy:.2f}, Recall: {recall:.2f}")

average_accuracy = np.mean(accuracies)
average_recall = np.mean(recalls)

print(f"Average Accuracy across {num_iterations} iterations: {average_accuracy:.2f}")
print(f"Average Recall across {num_iterations} iterations: {average_recall:.2f}")

Iteration 1 - Accuracy: 0.93, Recall: 0.96
Iteration 2 - Accuracy: 0.94, Recall: 0.95
Iteration 3 - Accuracy: 0.94, Recall: 0.97
Average Accuracy across 3 iterations: 0.94
Average Recall across 3 iterations: 0.96
